In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%load_ext sql

In [ ]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'     #Sets the default table style (other options include 'MSWORD_FRIENDLY', 'PLAIN_COLUMNS', etc.)

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [7]:
!pip install -q pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%sql sqlite:///my_data1.db

In [4]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [9]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [10]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [11]:
unique_launch_sites = df['Launch_Site'].unique()
print(unique_launch_sites)

['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']


In [ ]:
#show only the rows where the 'Launch_Site' column values start with the string 'CCA'
df[df['Launch_Site'].str.startswith('CCA')].head(5)

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [13]:
total_payload_mass = df[df['Customer'] == 'NASA (CRS)']['PAYLOAD_MASS__KG_'].sum()
print("Total payload mass carried by boosters launched by NASA (CRS):", total_payload_mass, "kg")

Total payload mass carried by boosters launched by NASA (CRS): 45596 kg


In [14]:
avg_payload_f9_v1_1 = df[df['Booster_Version'] == 'F9 v1.1']['PAYLOAD_MASS__KG_'].mean()
print("Average payload mass carried by booster version F9 v1.1:", avg_payload_f9_v1_1, "kg")

Average payload mass carried by booster version F9 v1.1: 2928.4 kg


In [15]:
first_ground_pad_landing = df[df['Landing_Outcome'] == 'Success (ground pad)']['Date'].min()
print("Date of first successful landing outcome in ground pad:", first_ground_pad_landing)

Date of first successful landing outcome in ground pad: 2015-12-22


In [16]:
boosters = df[
    (df['Landing_Outcome'] == 'Success (drone ship)') &
    (df['PAYLOAD_MASS__KG_'] > 4000) &
    (df['PAYLOAD_MASS__KG_'] < 6000)
]['Booster_Version'].unique()
print("Boosters with success in drone ship and payload mass between 4000 and 6000 kg:", boosters)

Boosters with success in drone ship and payload mass between 4000 and 6000 kg: ['F9 FT B1022' 'F9 FT B1026' 'F9 FT  B1021.2' 'F9 FT  B1031.2']


In [17]:
success_count = df[df['Mission_Outcome'].str.lower().str.contains('success')].shape[0]
failure_count = df[df['Mission_Outcome'].str.lower().str.contains('failure')].shape[0]
print("Total successful mission outcomes:", success_count)
print("Total failure mission outcomes:", failure_count)

Total successful mission outcomes: 100
Total failure mission outcomes: 1


In [ ]:
query = """
SELECT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL
)
"""
result = pd.read_sql_query(query, con)
print(result['Booster_Version'].unique())

['F9 B5 B1048.4' 'F9 B5 B1049.4' 'F9 B5 B1051.3' 'F9 B5 B1056.4'
 'F9 B5 B1048.5' 'F9 B5 B1051.4' 'F9 B5 B1049.5' 'F9 B5 B1060.2 '
 'F9 B5 B1058.3 ' 'F9 B5 B1051.6' 'F9 B5 B1060.3' 'F9 B5 B1049.7 ']


In [19]:
query_2015_failures = """
SELECT 
    CASE substr(Date, 6, 2)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS Month_Name,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTBL
WHERE substr(Date, 0, 5) = '2015'
  AND Landing_Outcome LIKE '%Failure (drone ship)%'
"""
failures_2015 = pd.read_sql_query(query_2015_failures, con)
print(failures_2015)

  Month_Name       Landing_Outcome Booster_Version  Launch_Site
0    January  Failure (drone ship)   F9 v1.1 B1012  CCAFS LC-40
1      April  Failure (drone ship)   F9 v1.1 B1015  CCAFS LC-40


In [20]:
landing_counts = df[
    (df['Date'] >= '2010-06-04') & (df['Date'] <= '2017-03-20')
]['Landing_Outcome'].value_counts().sort_values(ascending=False)
print(landing_counts)

Landing_Outcome
No attempt                10
Failure (drone ship)       5
Success (drone ship)       5
Controlled (ocean)         3
Success (ground pad)       3
Failure (parachute)        2
Uncontrolled (ocean)       2
Precluded (drone ship)     1
Name: count, dtype: int64
